# Ejercicio Strem Cipher
Marco A. Jurado 20308

## Keystream

## Cifrado

## Descifrado

## Preguntas
#### ¿Cómo cambia el mensaje cifrado cuando cambias la clase?

#### ¿Qué sucede si utilizamos el mismo keystream para cifrar dos mensajes diferentes?

#### ¿Cómo afecta la longitus del keystream a la seguridad del cifrado?